In [ ]:
from doc2vec import read_fasta_to_kmers, encode_and_labels
from model import train_doc2vec_model
from model_tools import learning_curve, evalution_metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# load data
train_pos = read_fasta_to_kmers('../data/afp_pos_seq_len50train_2710.fasta')
train_neg = read_fasta_to_kmers('../data/afp_neg_seq_len50train_2710.fasta')
test_pos = read_fasta_to_kmers('../data/afp_pos_seq_len50independant_test_301.fasta')
test_neg = read_fasta_to_kmers('../data/afp_neg_seq_len50independant_test_301.fasta')

In [ ]:
# encoding through Doc2Vec pretrained 
doc2vec_model = '../Doc2Vec_model/AFP_doc2vec.model'
train_data, train_labels = encode_and_labels(train_pos, train_neg, doc2vec_model)
test_data, test_labels = encode_and_labels(test_pos, test_neg, doc2vec_model)

In [ ]:
train_data=train_data.reshape((train_data.shape[0],train_data.shape[1],1))
test_data=test_data.reshape((test_data.shape[0],test_data.shape[1],1))

In [ ]:
train_data_, test_data_, train_labels_, test_labels_ = train_test_split(train_data, train_labels, test_size= 0.1, random_state = 10, stratify = train_labels)


## CNN model training

In [ ]:
t_m = train_doc2vec_model(train_data_, train_labels_, test_data_, test_labels_, 'doc2vec', path = '../Doc2Vec_model')

In [ ]:
learning_curve(t_m.history)

In [ ]:
model = load_model('../Doc2Vec_model/doc2vec_best_weights.h5')
labels_score = model.predict(test_data)
evalution_metrics(test_labels, labels_score)

## 10-fold cross-validation

In [ ]:
from sklearn.model_selection import KFold

def fold_cv(train_data, labels, model_name=None, output_dir = '.'):
    if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
    kfold = KFold(n_splits=10, shuffle=True)
    # K-fold Cross Validation model evaluation
    df = pd.DataFrame(columns=['accuracy', 'precision', 'sensitivity', 'specificity', 'f1', 'mcc'])
    fold_no = 1
    for train, val in kfold.split(train_data, labels):
        # Generate a print
        print('------------------------------------------------------------------------')
        print('Training for fold {fold_no} ...')
        
        train_doc2vec_model(train_data[train], labels[train], train_data[val], labels[val], model_name = 'kfold%s'%fold_no, path = output_dir)
        model = load_model(os.path.join(output_dir, 'kfold%s_best_weights.h5'%fold_no))
        
        labels_score = model.predict(train_data[val])
        metrics_dict = evalution_metrics(labels[val], labels_score, save=False)
        print(metrics_dict)
        df.loc[fold_no] = metrics_dict.values()
        # Increase fold number
        fold_no = fold_no + 1
    df.loc['Mean'] = df.mean()
    df.to_csv(os.path.join(output_dir,'%s_cv.csv'%model_name))
    return(df)

In [ ]:
fold_cv(train_data, train_labels, output_dir = '../Doc2Vec_model/10_fold')